In [1]:
using Random, CategoricalArrays, DataFrames, DataFramesMeta, StatsBase, CSV, ScikitLearn

In [2]:
data_path_to_load = "C:/Users/15125/Desktop/CAPSTONE PROJECT/CLEANED DATA/FAIRNESS MODEL/"
weighted_sev = CSV.read(data_path_to_load*"weighted_severity_penetrating_0.0001.csv")
changed_label = weighted_sev[!,:changed_labels]
IJulia.clear_output()

0

In [3]:
function oct_train(train_X, train_y, test_X, test_y)
    oct = IAI.OptimalTreeClassifier(random_seed=1, criterion=:gini, 
        missingdatamode=:separate_class, treat_unknown_level_missing=true, 
        ls_num_tree_restarts= 60, minbucket= 100)
    grid_oct = IAI.GridSearch(oct,max_depth=5)
    IAI.fit_cv!(grid_oct, train_X, train_y, n_folds=5, sample_weight=:autobalance)
    best_params_grid = IAI.get_best_params(grid_oct)
    best_OCT = IAI.get_learner(grid_oct)
    y_pred_train= IAI.predict(best_OCT, train_X)
    y_pred_test= IAI.predict(best_OCT, test_X)
    non_no_change_obs_train = train_y .!= "no change"
    non_no_change_obs_test = test_y .!= "no change"
    ins_acc = IAI.score(best_OCT, train_X, train_y, criterion=:misclassification)
    oos_acc = IAI.score(best_OCT, test_X, test_y,  criterion=:misclassification)
    ins_acc2 = IAI.score(best_OCT, train_X[non_no_change_obs_train,:], train_y[non_no_change_obs_train], criterion=:misclassification)
    oos_acc2 = IAI.score(best_OCT, test_X[non_no_change_obs_test,:],  test_y[non_no_change_obs_test],  criterion=:misclassification)
    return ins_acc, oos_acc, ins_acc2, oos_acc2, best_OCT
end

oct_train (generic function with 1 method)

In [4]:
function pre_process_and_train(X, y, seed_num)
    (train_X, train_y), (test_X, test_y) = IAI.split_data(:classification, X, y, seed= seed_num)
    
    severity_categorical_var = [
    :Head_severity,
    :Face_severity,
    :Neck_severity,
    :Thorax_severity,
    :Abdomen_severity,
    :Spine_severity,
    :Upper_Extremity_severity,
    :Lower_Extremity_severity,
    :Pelvis_Perineum_severity,
    :External_severity,
    :gcstot1,
    #:severity_max
    ]
    
    for col_name in severity_categorical_var
        train_X[!, col_name] = CategoricalArray(train_X[!, col_name], ordered=true)
        test_X[!, col_name] = CategoricalArray(test_X[!, col_name], ordered=true)  
        println("Transformed column '", col_name, "' to Ordered Categorical")
    end
    
    train_X = categorical!(train_X)
    test_X = categorical!(test_X)
    println("Transformed all other non-ordered Categorical Variables")
    
    names = [
        :age, :gender, :signsoflife, :eddisp, :sbp1, :pulse1,
        :oxysat1, :temp1, :gcstot1, :bleeding_disorder, :current_chemotherapy,
        :congestive_heart_failure, :current_smoker, :chronic_renal_failure,
        :history_cva, :diabetes, :disseminated_cancer,  :copd, :steroid,
        :cirrhosis, :history_MI, :history_pvd, :hypertension_medication,
        :method_of_injury, :Head_severity, :Face_severity, :Neck_severity, :Thorax_severity, :Abdomen_severity,
        :Spine_severity, :Upper_Extremity_severity, :Lower_Extremity_severity,
        :Pelvis_Perineum_severity, :External_severity,:alcohol_use_disorder,:drug_use_disorder,:rr1]
    
    train_X = train_X[!, names]
    test_X = test_X[!, names]
    
    ins_acc, oos_acc, ins_acc2, oos_acc2, best_OCT = oct_train(train_X, train_y, test_X, test_y)
    
    return ins_acc, oos_acc, ins_acc2, oos_acc2, best_OCT 
end

pre_process_and_train (generic function with 1 method)

## Changed Labels

In [5]:
ins_accs = []
ins_accs2 = []
oos_accs = []
oos_accs2 = []

for num in 1:10
    println("Beginning run number ", num)
    println("\n")
    ins_acc, oos_acc, ins_acc2, oos_acc2, best_OCT = pre_process_and_train(weighted_sev, changed_label, num)
    ins_accs = append!(ins_accs, ins_acc)
    ins_accs2 = append!(ins_accs2, ins_acc2)
    oos_accs = append!(oos_accs, oos_acc)
    oos_accs2 = append!(oos_accs2, oos_acc2)
    IAI.write_json("Fairness_OCT_change_of_label_no_race $num.json",best_OCT)
    println("Ending run number ", num)
    println("\n")
end

Beginning run number 1


Transformed column 'Head_severity' to Ordered Categorical
Transformed column 'Face_severity' to Ordered Categorical
Transformed column 'Neck_severity' to Ordered Categorical
Transformed column 'Thorax_severity' to Ordered Categorical
Transformed column 'Abdomen_severity' to Ordered Categorical
Transformed column 'Spine_severity' to Ordered Categorical
Transformed column 'Upper_Extremity_severity' to Ordered Categorical
Transformed column 'Lower_Extremity_severity' to Ordered Categorical
Transformed column 'Pelvis_Perineum_severity' to Ordered Categorical
Transformed column 'External_severity' to Ordered Categorical
Transformed column 'gcstot1' to Ordered Categorical
Transformed all other non-ordered Categorical Variables


CV fold...                           0%|█               |  ETA: N/A┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ IAILicensing C:\Users\iai\builds\InterpretableAI\SysImgBuilder\.julia\packages\IAILicensing\x1CT6\src\precompile.jl:19

2mTraining trees...                    0%|█               |  ETA: N/A
2mTraining trees...                    2%|█               |  ETA: 0:05:34
2mTraining trees...                    3%|█               |  ETA: 0:03:22
2mTraining trees...                    5%|█               |  ETA: 0:02:41
2mTraining trees...                    7%|██              |  ETA: 0:02:29
2mTraining trees...                    8%|██              |  ETA: 0:02:17
2mTraining trees...                   10%|██              |  ETA: 0:02:10
2mTraining trees...                   12%|██              |  ETA: 0:02:01
2mTraining trees...                   13%|███             |  ETA: 0:01:54
2mTraining trees...                   1

2mTraining trees...                   58%|██████████      |  ETA: 0:00:34
2mTraining trees...                   60%|██████████      |  ETA: 0:00:32
2mTraining trees...                   62%|██████████      |  ETA: 0:00:31
2mTraining trees...                   63%|███████████     |  ETA: 0:00:29
2mTraining trees...                   65%|███████████     |  ETA: 0:00:28
2mTraining trees...                   67%|███████████     |  ETA: 0:00:26
2mTraining trees...                   68%|███████████     |  ETA: 0:00:25
2mTraining trees...                   70%|████████████    |  ETA: 0:00:24
2mTraining trees...                   72%|████████████    |  ETA: 0:00:23
2mTraining trees...                   73%|████████████    |  ETA: 0:00:21
2mTraining trees...                   75%|█████████████   |  ETA: 0:00:20
2mTraining trees...                   77%|█████████████   |  ETA: 0:00:19
2mTraining trees...                   78%|█████████████   |  ETA: 0:00:18
2mTraining trees...                   

2mTraining trees...                   22%|████            |  ETA: 0:00:49
2mTraining trees...                   23%|████            |  ETA: 0:00:48
2mTraining trees...                   25%|█████           |  ETA: 0:00:46
2mTraining trees...                   27%|█████           |  ETA: 0:00:45
2mTraining trees...                   28%|█████           |  ETA: 0:00:43
2mTraining trees...                   30%|█████           |  ETA: 0:00:44
2mTraining trees...                   32%|██████          |  ETA: 0:00:43
2mTraining trees...                   33%|██████          |  ETA: 0:00:42
2mTraining trees...                   35%|██████          |  ETA: 0:00:41
2mTraining trees...                   37%|██████          |  ETA: 0:00:41
2mTraining trees...                   38%|███████         |  ETA: 0:00:40
2mTraining trees...                   40%|███████         |  ETA: 0:00:38
2mTraining trees...                   42%|███████         |  ETA: 0:00:37
2mTraining trees...                   

2mTraining trees...                   85%|██████████████  |  ETA: 0:00:09
2mTraining trees...                   87%|██████████████  |  ETA: 0:00:08
2mTraining trees...                   88%|███████████████ |  ETA: 0:00:07
2mTraining trees...                   90%|███████████████ |  ETA: 0:00:06
2mTraining trees...                   92%|███████████████ |  ETA: 0:00:05
2mTraining trees...                   93%|███████████████ |  ETA: 0:00:04
2mTraining trees...                   95%|████████████████|  ETA: 0:00:03
2mTraining trees...                   97%|████████████████|  ETA: 0:00:02
2mTraining trees...                   98%|████████████████|  ETA: 0:00:01
Refitting with best parameters...    0%|█               |  ETA: N/A
4m  Parameters:  cp=>0.0009241 max_depth=>5

...

2mTraining trees...                   63%|███████████     |  ETA: 0:00:21
2mTraining trees...                   65%|███████████     |  ETA: 0:00:20
2mTraining trees...                   67%|███████████     |  ETA: 0:00:19
2mTraining trees...                   68%|███████████     |  ETA: 0:00:18
2mTraining trees...                   70%|████████████    |  ETA: 0:00:17
2mTraining trees...                   72%|████████████    |  ETA: 0:00:16
2mTraining trees...                   73%|████████████    |  ETA: 0:00:16
2mTraining trees...                   75%|█████████████   |  ETA: 0:00:15
2mTraining trees...                   77%|█████████████   |  ETA: 0:00:14
2mTraining trees...                   78%|█████████████   |  ETA: 0:00:13
2mTraining trees...                   80%|█████████████   |  ETA: 0:00:12
2mTraining trees...                   82%|██████████████  |  ETA: 0:00:11
2mTraining trees...                   83%|██████████████  |  ETA: 0:00:10
2mTraining trees...                   

2mTraining trees...                   27%|█████           |  ETA: 0:00:56
2mTraining trees...                   28%|█████           |  ETA: 0:00:56
2mTraining trees...                   30%|█████           |  ETA: 0:00:54
2mTraining trees...                   32%|██████          |  ETA: 0:00:53
2mTraining trees...                   33%|██████          |  ETA: 0:00:52
2mTraining trees...                   35%|██████          |  ETA: 0:00:50
2mTraining trees...                   37%|██████          |  ETA: 0:00:49
2mTraining trees...                   38%|███████         |  ETA: 0:00:48
2mTraining trees...                   40%|███████         |  ETA: 0:00:46
2mTraining trees...                   42%|███████         |  ETA: 0:00:45
2mTraining trees...                   43%|███████         |  ETA: 0:00:43
2mTraining trees...                   45%|████████        |  ETA: 0:00:41
2mTraining trees...                   47%|████████        |  ETA: 0:00:39
2mTraining trees...                   

2mTraining trees...                   90%|███████████████ |  ETA: 0:00:06
2mTraining trees...                   92%|███████████████ |  ETA: 0:00:05
2mTraining trees...                   93%|███████████████ |  ETA: 0:00:04
2mTraining trees...                   95%|████████████████|  ETA: 0:00:03
2mTraining trees...                   97%|████████████████|  ETA: 0:00:02
2mTraining trees...                   98%|████████████████|  ETA: 0:00:01
Refitting with best parameters...    0%|█               |  ETA: N/A
4m  Parameters:  cp=>0.0006324 max_depth=>5



2mTraining trees...                    0%|█               |  ETA: N/A

2mTraining trees...                    2%|█               |  ETA: 0:02:30

2mTraining trees...                    3%|█               |  ETA: 0:01:39

2mTraining trees...                    5%|█               |  ETA: 0:01:30

2mTraining trees...                    7%|██              |  ETA: 0:01:25

2mTraining trees...                    8%|██              |  ETA: 0:01:20

2mTraining trees...                   10%|██              |  ETA: 0:01:21

2mTraining trees...                   12%|██              |  ETA: 0:01:18

2mTraining trees...                   13%|███             |  ETA: 0:01:15

2mTraining trees...                   15%|███             |  ETA: 0:01:15

2mTraining trees...                   17%|███             |  ETA: 0:01:12

2mTraining trees...                   18%|███             |  ETA: 0:01:10

2mTraining trees...                   20%|████            |  ETA: 0:01:09

2mTraining trees...        




Ending run number 10




In [6]:
results = DataFrame(
    ins_acc_train = ins_accs,
    ins_acc_test = ins_accs2,
    oos_acc_train = oos_accs,
    oos_acc_test = oos_accs2,
)

,ins_acc_train,ins_acc_test,oos_acc_train,oos_acc_test
,Any,Any,Any,Any
1,0.138999,0.968485,0.136522,0.966102
2,0.377161,0.884242,0.379836,0.837571
3,0.400686,0.881212,0.400038,0.834746
4,0.389223,0.87697,0.387587,0.860169
5,0.40025,0.867879,0.405057,0.864407
6,0.407902,0.877576,0.406645,0.847458
7,0.397228,0.865455,0.394384,0.878531
8,0.379176,0.87697,0.375135,0.84887
9,0.398535,0.881212,0.396481,0.850282


In [8]:
CSV.write("C:/Users/15125/Desktop/CAPSTONE PROJECT/CLEANED DATA/FAIRNESS MODEL/fairness_oct_split_change_of_label_results_depth5_norace.csv", results)

"C:/Users/15125/Desktop/CAPSTONE PROJECT/CLEANED DATA/FAIRNESS MODEL/fairness_oct_split_change_of_label_results_depth5_norace.csv"